In [1]:
!nvidia-smi

Fri Sep 25 13:09:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -U fastai
!pip install lshashpy3
!pip install Bitarray

     |████████████████████████████████| 358kB 5.3MB/s 
     |████████████████████████████████| 40kB 4.8MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
  Created wheel for lshashpy3: filename=lshashpy3-0.0.8-cp36-none-any.whl size=8877 sha256=36f429f3a8e99a2d3905d68313aa4bc88298dfb0a0a149c9992dad4c22b7fb98
  Stored in directory: /root/.cache/pip/wheels/c6/81/29/a48985e27d56ddea4e601fda82b9be9962d20801b3cfa82c2d
Successfully built lshashpy3
     |████████████████████████████████| 61kB 2.8MB/s 
  Created wheel for Bitarray: filename=bitarray-1.5.3-cp36-cp36m-linux_x86_64.whl size=108807 sha256=f9c0e661c0c73d3a956b5fb5a129d83059d8624375783531c345dd2890ba8dd0
  Stored in directory: /root/.cache/pip/wheels/be/35/9d/2448d6e584a8a4980613c092fa2ef64b3df4ae6314089b0333
Successfully built Bitarray


In [3]:
from fastai.vision.all import *
import pickle
import lshashpy3
from tqdm import notebook
import PIL.Image
from numpy import load as np_load
import bitarray

In [4]:
#importing the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd drive/My\ Drive

/content/drive/My Drive


In [6]:
root_path = Path('.')
data_path = Path('data_512')
model_path = Path('Models')

In [8]:
learner = load_learner(model_path/"archeo_resnet50.pkl")

In [ ]:
class SaveFeatures:
    features = None

    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module, input, output):
        out = output.detach().cpu().numpy()
        if isinstance(self.features, type(None)):
            self.features = out
        else:
            self.features = np.row_stack((self.features, out))

    def remove(self):
        self.hook.remove()

sf = SaveFeatures(learner.model[1][5])

In [ ]:
lsh = pickle.load(open(model_path/"lsh.p", "rb"))

In [ ]:
print(lsh.query())

TypeError: ignored

In [ ]:
def image_to_features(img, hook, learner):
    '''
    Function to convert image to feature vector
    '''
    _ = learner.predict(img)
    vect = hook.features[-1]
    return vect

In [ ]:
def get_similar_images(img, conv_learn, hook, lsh, n_items):

    ## Converting Image to vector
    vect = image_to_features(img, hook, conv_learn)
    print(len(vect))

    ## Finding approximate nearest neighbours using LSH
    response = lsh.query(vect, num_results=n_items, distance_func="hamming")
    print(response)

    path_to_result_images = []
    for k in range(n_items):
      path_to_result_images += [response[k][0][1]]

    return path_to_result_images


In [ ]:
def save_result(path_of_image_to_test, path_to_result_images):
    ## Dimension calculation for plotting
    n_items = len(path_to_result_images)
    columns = 3
    rows = int(np.ceil((n_items + 1 )/ columns))
    fig, axes = plt.subplots(rows, columns, figsize=(17,17))
    fig.tight_layout()
    fig.patch.set_facecolor('xkcd:navy')
    plt.setp(axes, xticks=[], yticks=[])
    plt.box(False)

    axes[0][1].imshow(Image.open(path_of_image_to_test), cmap='Greys_r')
    axes[0][0].patch.set_facecolor('xkcd:navy')
    axes[0][0].axis('off')
    axes[0][2].patch.set_facecolor('xkcd:navy')
    axes[0][2].axis('off')

    i = 0
    for row in range(1,rows):
      for col in range(columns):
        axes[row][col].imshow(Image.open(path_to_result_images[i]))
        axes[row][col].set_title(path_to_result_images[i][17:])
        # axes[row][col].title.set_color('white')
        i += 1

    plt.savefig('output/ambigues_transfo/' + path_of_image_to_test[len("ambigues_transfo"):])
    plt.close()

In [ ]:
files = os.listdir("ambigues_transfo")
for f in notebook.tqdm(files):
  # img = download_img_from_url("https://previews.123rf.com/images/elenka1/elenka11803/elenka1180301047/97767469-abstract-geometric-pattern-with-stripes-lines-a-seamless-vector-background-white-and-gold-ornament-.jpg")
  path_of_image_to_test = 'ambigues_transfo/' + f
  img_fastai = Path(path_of_image_to_test)

  path_to_result_images = get_similar_images(img_fastai, learner, sf, lsh, 3)
  save_result(path_of_image_to_test, path_to_result_images)


512
[]


IndexError: ignored